In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git  -U 
!pip install -q git+https://github.com/huggingface/accelerate.git  -U 
!pip install -q bitsandbytes 
!pip install -q git+https://github.com/huggingface/peft.git  -U 
!pip install -q langchain
!pip install -q sentence-transformers
!pip install -q faiss-gpu
!pip install requests

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2024.1.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2024.1.0 which is incompatible.
dask-cuda 23.8.0 require

# Extracting information from webpages

In [2]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import re
import time


def get_search_urls(query, num_results=5):
    """
    Retrieves URLs from a search engine for a given query.

    Args:
        query (str): The search query.
        search_engine (str): The search engine name (e.g., "google", "bing").
        num_results (int): The desired number of results (default: 20).

    Returns:
        list: A list of top URLs from the search.

    Raises:
        ValueError: If an invalid search engine is provided.
    """

    # Construct Google search URL with proper encoding
    for i in query:
        if i==" ":
            query+="+"
        else:
            query+=i
    print(query)
    query = "https://www.google.dz/search?q="+query
    page = requests.get(query)
    soup = BeautifulSoup(page.content,features="html.parser")
    url_list=[]
    count=0
    for link in soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
        if count == 0:
            count+=1
        else:
            url = re.split(":(?=http)",link["href"].replace("/url?q=",""))[0]
            url = re.sub(r"&sa.*", "", url)
            url_list.append(url)
    return url_list[:num_results]

In [3]:
def scrape_url(url):
    """
    Scrapes data from a given URL.

    Args:
        url (str): The URL to scrape.

    Returns:
        dict: A dictionary containing scraped data (format depends on website).
    """
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        text_list = []
        for i in ["p","code","answers","discussion_bucket","content_main"]:
            for p in soup.find_all(i):  # Replace "p" with relevant tags
                text_list.append(p.text.strip())  # Remove leading/trailing whitespace

        # Combine and clean text
        all_text = "\n".join(text_list)
        cleaned_text = re.sub(r"<.*?>", "", all_text)  # Remove HTML tags with regex
        count = 0
        final_text = ""
        for i in range(0,len(cleaned_text)):
            if cleaned_text[i]=="\n" and cleaned_text[i]=="\n":
                count+=1
                if count==200:
                    break
            final_text+=cleaned_text[i]
        return final_text
    except:
        return ""

In [4]:
def WebCrawler(query):
    # Get user input
    begin = time.time()

    # Retrieve top URLs
    search_urls = get_search_urls(query)

    txt_file_for_urls="/kaggle/working/URLlist.txt"
    with open(txt_file_for_urls, "w+") as file:  # Open in append mode
        for website in search_urls:
            try:
                data = scrape_url(website)  # Call your function
                # Combine and format data before saving
                formatted_data = f"Website: {website}\n{data}\n\n"  # Separate by website
                file.write(formatted_data)
            except Exception as e:
                print(f"Error scraping {website}: {e}")
    print("Time taken to execute: ", time.time()-begin)

In [5]:
get_search_urls("What is the best shampoo in India?")

What is the best shampoo in India?What+is+the+best+shampoo+in+India?


['https://www.newindianexpress.com/expressdeals/other-categories/beauty-and-health/best-shampoo-for-hair-growth-in-india',
 'https://www.newindianexpress.com/expressdeals/other-categories/beauty-and-health/best-shampoo-for-hair-growth-in-india',
 'https://www.google.comhttps://support.google.com/websearch%3Fp%3Dfeatured_snippets%26hl%3Den-US&opi=89978449&usg=AOvVaw2M8SCMBzjmvsKhurArc7_8',
 'https://www.vikatan.com/vikatan-deals/reviews/health-beauty/10-best-shampoos-for-hair-growth',
 'https://www.outlookindia.com/whats-hot/10-best-shampoos-for-dry-hair-in-india-2024']

# Initializing LLM

In [6]:
import torch
from transformers import BitsAndBytesConfig
# Create a quantization configuration using BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= True,
llm_int8_enable_fp32_cpu_offload= True)

In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",quantization_config=quantization_config)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

# Initializing Vector Database

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
import torch
from transformers import pipeline

2024-02-20 22:11:59.058417: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 22:11:59.058549: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 22:11:59.198186: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
# Define the model name for Hugging Face embeddings
model_name = 'WhereIsAI/UAE-Large-V1'

# Define model kwargs for Hugging Face embeddings during initialization
model_kwargs = {'device': 'cuda'}

# Define encode kwargs for Hugging Face embeddings
# Set 'normalize_embeddings' to True to compute cosine similarity
encode_kwargs = {'normalize_embeddings': True}

# Create Hugging Face embeddings instance
# Specify the model name, model kwargs, and encode kwargs
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

# Splitting the document

In [10]:
def docSplitter():
    # Create a TextLoader instance for loading text documents
    # Specify the path to the text file ("/kaggle/input/evspdf/RAG.txt" in this case)
    loader = TextLoader("/kaggle/working/URLlist.txt")

    # Load documents using the TextLoader
    documents = loader.load()
    # Create a RecursiveCharacterTextSplitter instance for splitting text documents
    # Set a small chunk size and overlap for demonstration purposes
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=500,
        length_function=len,
        is_separator_regex=False
    )

    # Split the loaded documents using the TextSplitter
    text_docs = text_splitter.split_documents(documents)
    
    return text_docs

# Context Function

In [11]:
def get_context(user_input_translated, db, k=1, fetch_k=200):
    '''
    Perform max marginal relevance search using the FAISS vector store
    Search for relevant documents based on the transliterated user input
    Usually you would want the fetch_k parameter >> k parameter
    This is because the fetch_k parameter is the number of documents that will be fetched before filtering. 
    If you set fetch_k to a low number, you might not get enough documents to filter from.
    k = Number of searches to be returned
    fetch_k = Number of documents that will be fetched before filtering
    '''
    docs = db.max_marginal_relevance_search(user_input_translated, k=k, fetch_k=fetch_k) 
    # Initialize an empty string to store concatenated contexts
    contexts = ""

    # Iterate through the retrieved documents and print page content
    for i, doc in enumerate(docs):
        # print(f"{i + 1}. {doc.page_content}")

        # Concatenate the page content for further processing if needed
        contexts += doc.page_content + "\n"
    
    return contexts

# Making text look better

In [12]:
import pathlib
import textwrap

# Import the generative AI module from the google package
import google.generativeai as genai

# Import necessary display modules from IPython
from IPython.display import display, Markdown

def to_markdown(text):
    """
    Convert plain text to Markdown format.

    This function takes a plain text input and converts it to Markdown format.
    It also replaces bullet points with proper Markdown list syntax.

    Args:
        text (str): The plain text to be converted to Markdown.

    Returns:
        Markdown: The converted Markdown text.

    Example:
        >>> plain_text = "This is a bullet point:\n• Item 1\n• Item 2"
        >>> markdown_output = to_markdown(plain_text)
        >>> print(markdown_output)
        > This is a bullet point:
        >   * Item 1
        >   * Item 2
    """
    # Replace special character '•' with proper Markdown list syntax
    text = text.replace('•', '  *')
    
    # Indent the text with '>' to format it as a blockquote
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Search Function

In [13]:
import time

In [16]:
def search(query):
    WebCrawler(query)
    text_docs = docSplitter()
    db = FAISS.from_documents(text_docs, embeddings)
    relevant_info = get_context(query,db)
    prompt = f"""<s>[INST] You are a helpful chatbot which uses the information provided to you help the users with their queries. Cite the websites used at the end.
### Information: {relevant_info}

### User Query: {query}
[/INST]"""
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
    start = time.time()
    sequences = pipe(
            f'{prompt}' ,
            do_sample=True,
            max_new_tokens=2048, 
            temperature=0.7, 
            top_p=0.95
        )
    extracted_title = re.sub(r'[\'"]', '', sequences[0]['generated_text'].split("[/INST]")[1])
    stop = time.time()
    time_taken = stop-start
    print(f"Execution Time : {time_taken}")
    return extracted_title
    

In [17]:
%%time
to_markdown(search("How do i install Linux on my external SSD step by step?"))

How do i install Linux on my external SSD step by step?How+do+i+install+Linux+on+my+external+SSD+step+by+step?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Time taken to execute:  1.9485552310943604
Execution Time : 59.330870151519775
CPU times: user 59.9 s, sys: 70.1 ms, total: 60 s
Wall time: 1min 1s


>  To install Linux on an external SSD, you can follow the steps below. These instructions are based on the article from Medium (<https://medium.com/geekculture/installing-linux-ubuntu-20-04-on-an-external-portable-ssd-and-pitfalls-to-be-aware-of-388294e701b5>) and some additional resources.
> 
> 1. **Preparation:**
>    a. Backup all your data on the external SSD.
>    b. Install Ubuntu Desktop on a separate internal drive or a virtual machine if you dont have it already.
>    c. Connect the SSD to your computer.
> 
> 2. **Create a bootable USB:**
>    a. Install `Rufus` or `Unetbootin` on your Ubuntu system.
>    b. Use the software to create a bootable USB stick from the Ubuntu ISO file.
> 
> 3. **Partition the SSD:**
>    a. Open the Ubuntu installer using the bootable USB stick.
>    b. In the Install Ubuntu window, select your language and click Continue.
>    c. In the Disk usage screen, select the SSD and click Install Now.
>    d. In the Partition Disks window, click New Partition Table.
>    e. Create a new partition, mount point, and file system for your SSD. Be sure to allocate enough space for the root, home, swap, and boot partitions.
>    f. Click Install Now and follow the on-screen instructions to complete the installation.
> 
> 4. **Boot from the SSD:**
>    a. Restart your computer and change the boot order in the BIOS settings to prioritize the SSD.
>    b. If your computer doesnt boot from the SSD, you may need to modify the GRUB configuration file or use a bootloader like `rEFInd`.
> 
> 5. **Install additional software:**
>    a. Use the terminal to install additional software and drivers as needed.
> 
> Some potential pitfalls to be aware of when installing Linux on an external SSD include:
> 
> - External SSDs may not support hibernation or suspend-to-disk features.
> - Depending on your hardware, you might face issues with power management and battery life when using the SSD as the main boot drive.
> - Certain applications may not run optimally on an external SSD due to performance limitations.
> 
> For more information and troubleshooting, refer to the article on Medium and the resources linked in this response.
> 
> [Medium](https://medium.com/geekculture/installing-linux-ubuntu-20-04-on-an-external-portable-ssd-and-pitfalls-to-be-aware-of-388294e701b5), [Google Search: Featured snippets](https://support.google.com/websearch%3Fp%3Dfeatured_snippets%26hl%3Den-US&opi=89978449&usg=AOvVaw2M8SCMBzjmvsKhurArc7_8), [YouTube: Installing Ubuntu on an external SSD](https://www.youtube.com/watch?v=bfWdnCIrcxk), [YouTube: Installing Linux on an external SSD](https://www.youtube.com/watch?v=bfWdnCIrcxk)